In [23]:
import tweepy
import requests
import datetime
import time
import os
from dotenv import load_dotenv
import pytz

# Load environment variables
load_dotenv(dotenv_path="/Users/hodanielkhuu/Library/CloudStorage/OneDrive-HøyskolenKristiania/Personal/Annet/Twitter Monitor/.env")

# Get credentials from environment variables
BEARER_TOKEN = os.getenv('BEARER_TOKEN')
DISCORD_WEBHOOK_URL = os.getenv('DISCORD_WEBHOOK_URL')

# Authenticate using Bearer Token for Twitter API V2
client = tweepy.Client(bearer_token=BEARER_TOKEN)

# List of usernames to monitor
usernames = ['cmooonnfuuuuu']

# Convert usernames to user IDs
user_ids = []
for username in usernames:
    try:
        user = client.get_user(username=username)
        if user.data:
            user_ids.append(user.data.id)
            print(f"User ID for {username}: {user.data.id}")
        else:
            print(f"Couldn't find user: {username}")
    except Exception as e:
        print(f"Error finding user {username}: {e}")

# Define the maximum number of tweets to retrieve per user
max_tweets = 10  # Between 5 and 100

# Dictionary to store the most recent tweet ID for each user
last_tweet_times = {user_id: datetime.datetime.now(pytz.UTC) - datetime.timedelta(minutes=1) for user_id in user_ids}

# Function to post tweet to Discord via Webhook
def post_to_discord(tweet_text, tweet_url, username):
    # Create a more informative message with tweet content preview
    preview = tweet_text[:100] + "..." if len(tweet_text) > 100 else tweet_text
    
    data = {
        "content": f"**New tweet from @{username}**\n{preview}\n{tweet_url}"
    }
    
    response = requests.post(DISCORD_WEBHOOK_URL, json=data)
    if response.status_code == 204:
        print(f"Tweet successfully posted to Discord: {tweet_url}")
        return True
    else:
        print(f"Failed to post tweet. Status code: {response.status_code}")
        return False

# Function to fetch and process new tweets
def fetch_new_tweets(user_id):
    try:
        # Get the last processed tweet time for this user
        last_time = last_tweet_times[user_id]
        
        # Fetch recent tweets
        response = client.get_users_tweets(
            id=user_id, 
            max_results=max_tweets, 
            tweet_fields=["created_at", "text", "id"],
            exclude=["retweets", "replies"]  # Optional: exclude retweets and replies
        )
        
        # If no tweets are found
        if not response.data:
            print(f"No tweets found for user ID {user_id}.")
            return 0
        
        # Get user info for the username
        user_info = client.get_user(id=user_id)
        username = user_info.data.username if user_info.data else "unknown"
        
        # Track the newest tweet time we've seen
        newest_time = last_time
        new_count = 0
        
        # Process tweets in reverse order (oldest first) to maintain chronological order
        for tweet in reversed(response.data):
            created_at = tweet.created_at.astimezone(pytz.UTC)
            
            # Only process tweets newer than the last one we processed
            if created_at > last_time:
                tweet_url = f"https://twitter.com/{username}/status/{tweet.id}"
                print(f"New tweet from {username} at {created_at}: {tweet_url}")
                
                # Post to Discord
                if post_to_discord(tweet.text, tweet_url, username):
                    new_count += 1
                
                # Update the newest time if this tweet is newer
                if created_at > newest_time:
                    newest_time = created_at
        
        # Update the last tweet time for this user
        if newest_time > last_time:
            last_tweet_times[user_id] = newest_time
            print(f"Updated last tweet time for {username} to {newest_time}")
        
        return new_count
        
    except tweepy.TooManyRequests:
        print(f"Rate limit exceeded. Sleeping for 15 minutes...")
        time.sleep(15 * 60)
        return 0
    except Exception as e:
        print(f"Error fetching tweets: {e}")
        return 0

# Function to continuously poll for new tweets
def continuously_poll_for_tweets():
    print("Starting Twitter monitor...")
    
    # Print the current time when starting
    current_time = datetime.datetime.now(pytz.UTC)
    print(f"Monitor started at {current_time} UTC")
    print(f"Monitoring {len(user_ids)} Twitter accounts")
    
    # Initial wait to give Twitter API time to index new tweets
    time.sleep(5)
    
    while True:
        total_new_tweets = 0
        
        try:
            for user_id in user_ids:
                print(f"Checking for new tweets from user ID {user_id}...")
                new_tweets = fetch_new_tweets(user_id)
                total_new_tweets += new_tweets
                
                # Short sleep between users to avoid rate limits
                time.sleep(2)
            
            # Log activity
            if total_new_tweets > 0:
                print(f"Found and posted {total_new_tweets} new tweets")
            else:
                print("No new tweets found in this polling cycle")
            
            # Sleep between polling cycles
            interval = 60  # Check every minute
            print(f"Waiting {interval} seconds before next check...")
            time.sleep(interval)
            
        except Exception as e:
            print(f"Unexpected error: {e}")
            print("Continuing in 60 seconds...")
            time.sleep(60)

# Start the monitoring process
if __name__ == "__main__":
    if not user_ids:
        print("No valid user IDs found. Please check the usernames.")
    elif not BEARER_TOKEN:
        print("Missing Twitter API Bearer Token. Check your .env file.")
    elif not DISCORD_WEBHOOK_URL:
        print("Missing Discord Webhook URL. Check your .env file.")
    else:
        continuously_poll_for_tweets()

User ID for cmooonnfuuuuu: 1771319317205757952
Starting Twitter monitor...
Monitor started at 2025-02-26 17:32:18.331599+00:00 UTC
Monitoring 1 Twitter accounts
Checking for new tweets from user ID 1771319317205757952...
No tweets found for user ID 1771319317205757952.
No new tweets found in this polling cycle
Waiting 60 seconds before next check...
Checking for new tweets from user ID 1771319317205757952...
Rate limit exceeded. Sleeping for 15 minutes...


KeyboardInterrupt: 